#### 네이버 검색 결과 크롤링 - 수원 맛집

In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd

- requests.get()으로 데이터를 가자져 온경우

In [13]:
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='+quote('수원맛집')
res =requests.get(url)
soup =BeautifulSoup(res.text)

In [14]:
lis = soup.select('.bx._svp_item')
len(lis)

30

- selenium을 이용해서 데이터 가져오기

In [15]:
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

driver =webdriver.Chrome('file:///C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver')
driver.get(url)

In [16]:
# View 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)


In [17]:
soup =BeautifulSoup(driver.page_source)
lis =soup.select('.bx._svp_item')
len(lis)

30

In [18]:
driver.close()

- 날짜 ,제목,내용 크롤링

In [19]:
li =lis[0]
date=li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2023.04.05.'

In [27]:
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'일품이던 수원맛집베스트10'

In [32]:
content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
content[:200]

'하러 수원맛집베스트10에 속한다는 갈비집을 다녀왔어요. 동네 주민들 뿐만 아니라 멀리서도 많은 사람들이 찾아가는 곳이라는데 소문대로 고퀄리티의 식사를 즐길 수 있던 곳이었어요. 고반가든 서수원점 주소 : 경기...'

- 날짜 처리

In [23]:
from datetime import datetime,timedelta

today = datetime.now()
today

datetime.datetime(2023, 4, 25, 12, 12, 2, 942162)

In [26]:
# 어제
date=today -timedelta(1)
f'{date.year}.{date.month:02d}.{date.day}'

'2023.4.24'

In [29]:
#2일전
date=today -timedelta(2)
f'{date.year}.{date.month:02d}.{date.day}'

'2023.4.23'

In [33]:
# 23시간전
date = today - timedelta(hours=23)
date.strftime('%Y.%m.%d')

'2023.04.24'

In [34]:
#45분전
date = today - timedelta(minutes=45)
date.strftime('%Y.%m.%d')

'2023.04.25'

In [38]:
def conver_date(s):
    if len(s) == 11:    # '2023.04.25.' 10글자
        return s[:-1]
    today = datetime.now()
    if '어제' in s:
        date = today - timedelta(1)
    elif '일' in s:
        date = today - timedelta(int(s[0]))
    elif '시간' in s:
        date =  today - timedelta(hours=int(s[:-4]))
    elif '분' in s:
        date =  today - timedelta(minutes=int(s[:-3])) 

    return date.strftime('%Y.%m.%d')


- 데이터 프레임 만들기


In [39]:
lines = []
for li in lis:
    date=li.select_one('.sub_time.sub_txt').get_text().strip()
    date = conver_date(date)
    title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    lines.append({'날짜':date,'제목':title,'내용':content})

In [40]:
df=pd.DataFrame(lines)
df.tail()

,날짜,제목,내용
25,2023.04.21,수원 맛집 우세 신기한 메뉴가 가득한 분위기 좋은 영통 술집,곳이 수원에 생기고여 말았네요. 이미 시작부터 맛집 등극할 것만 같은 그런 느낌이라...
26,2023.04.10,"수원맛집 인계동맛집/ 24시간 영업하는 매력있는 평양냉면집, ""평장원""","내 나이 OO세가 되었는데, 감정적 기복은 여전하다. 이제야 깨닫는 진리는...나이..."
27,2023.04.16,동탄 수원 맛집 [우판등심] 숙성한우등심 가성비맛집,하루되세요 #내돈내산 #내돈내먹 #동탄한우 #동탄소고기 #수원한우 #수원소고기 #매...
28,2023.04.03,"수원 맛집 행궁동 맛집 백스트리트 피자, 피맥맛집 행궁동 술집으로 추천",수원맛집 #행궁동맛집 #백스트리트피자 #포테이토피자 #행궁동피맥맛집 #행궁동피자 #...
29,2023.04.03,영통구청 맛집 수원뭉티기 반해버린 뭉티기한판 수원 맛집,다음엔 나도 전통주 도전!! 수원 뭉티기 월~토 16:00 ~ 익일 01:00 일요...
